In [2]:
import csv
import base64
import time
import fnmatch
import urllib
import traceback
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored

import requests
from bs4 import BeautifulSoup

In [3]:
def crawl_dependents(repo,page_num):
    url = 'https://github.com/{}/network/dependents'.format(repo)
    dependents_data = []
    list_end = False
    
    for i in range(page_num):
        #print("GET " + url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        page_data = [
            "{}/{}".format(
                t.find('a', {"data-repository-hovercards-enabled":""}).text,
                t.find('a', {"data-hovercard-type":"repository"}).text
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        
        for dependent in page_data:
            if dependent in dependents_data:
                list_end = True 
        
        if list_end:
            break
        else:    
            dependents_data = dependents_data + page_data
        
        try:
            paginationContainer = soup.find("div", {"class":"paginate-container"}).find_all('a')
        except:
            break
        
        try:
            if len(paginationContainer) > 1:
                paginationContainer = paginationContainer[1]
            else:
                paginationContainer = paginationContainer[0]
        except:
            break
        
        if paginationContainer:
            url = paginationContainer["href"]
        else:
            break
        
    return dependents_data

In [4]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = "{:02d}:{:02d}".format(mins, secs)
        print(timeformat, end="\r")
        time.sleep(1)
        t -= 1
    print("\n\n\n\n\n")

In [5]:
assert load_dotenv(), 'Environment variables could not be loaded'
g = Github(getenv("GITHUB"))

In [6]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(
    urllib.parse.quote(filename)
).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open(".awesome.md", "w")
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Datasets APIs and open source projects related to Climate Change - A curated list of APIs, open data and ML/AI projects on climate change. [https://github.com/KKulma/climate-change-data] - Awesome Green Software - Research, tools, code, libraries and training to for building applications that emit less carbon into our atmosphere. [https://github.com/Green-Software-Foundation/awesome-green-software] - Awesome Sustainability Jobs - A curated list of companies in the sustainability sector that have jobs for devs. [https://github.com/pogopaule/awesome-sustainability-jobs] - Awesome Spectral Indices - A ready-to-use curated list of Spectral Indices for Remote Sensing applications. [https://github.com/davemlz/awesome-ee-spectral-indices] - Awesome Vegetation Index - List of reference, applications of common Vegetation Indices for Multi-spectral, hyper-spectral and UAV images. [https://github.com/px39n/Awesome-Vegetation-Index] - awesome-transit - Community list of transit APIs, apps, data

In [7]:
csv_github_projects = open("./csv/github_projects.csv", "w", newline="")
csv_github_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "topics",
    "rubric",
    "last_update",
    "stargazers_count",
    "number_of_dependents",
    "stars_last_year",
    "project_active_checkbox",
    "dominating_language",
    "organization",
    "organization_user_name",
    "languages",
    "homepage",
    "created",
    "license",
    "total_commits_last_year",
    "total_number_of_commits",
    "last_issue_closed",
    "open_issues",
    "closed_pullrequests",
    "closed_issues",
    "open_pullrequests",
    "reviews_per_pr",
    "community_development_distribution_score",
    "last_released_date",
    "last_release_tag_name",
    "good_first_issue",
    "contributors",
    "accepts_donations",
    "donation_platforms",
    "code_of_conduct",
    "contribution_guide",
    "organization_avatar",
    "dependents_repos"

]
writer_github_projects = csv.DictWriter(
    csv_github_projects, fieldnames=csv_github_fieldnames
)
writer_github_projects.writeheader()

csv_github_organizations = open("./csv/github_organizations.csv", "w", newline="")
csv_github_organizations_fieldnames = [
    "organization_name",
    "organization_user_name",
    "organization_github_url",
    "organization_website",
    "organization_location",
    "organization_avatar",
    "organization_public_repos",
    "organization_created",
    "organization_last_update",
]

                            
writer_github_organizations = csv.DictWriter(
    csv_github_organizations, fieldnames=csv_github_organizations_fieldnames
)
writer_github_organizations.writeheader()
organization_list = []

csv_gitlab_projects = open("./csv/gitlab_projects.csv", "w", newline="")
csv_gitlab_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_gitlab_projects = csv.DictWriter(
    csv_gitlab_projects, fieldnames=csv_gitlab_fieldnames
)
writer_gitlab_projects.writeheader()

csv_custom_platform_projects = open(
    "./csv/custom_platform_projects.csv", "w", newline=""
)
csv_custom_platform_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_custom_platform_projects = csv.DictWriter(
    csv_custom_platform_projects, fieldnames=csv_custom_platform_fieldnames
)
writer_custom_platform_projects.writeheader()

52

In [8]:
for r in repo_dict.rubrics:
    for entry in r.entries:
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == "github.com":
            print("%s is a GitHub project" % entry.name)
            try:

                remaining, limit = g.rate_limiting
                resettime = g.rate_limiting_resettime
                min_requests = 300
                if remaining < min_requests:
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp()
                    countdown((int(resettime) - int(current_time) + 2))

                print(
                    "GitHub Requests | Limit: {}, Remaining: {}".format(
                        limit, remaining
                    )
                )
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(entry.url).path.strip("/")
                user, project_name = path.split(repo_path)
                repo = g.get_repo(repo_path)
                contents_root = repo.get_contents("")
                releases = repo.get_releases()
                commits = repo.get_commits()
                stargazers = repo.get_stargazers_with_dates()
                
                # Crawel dependents
                try:
                    dependents_repos = crawl_dependents(repo_path,20)
                    
                except Exception as e:
                    print("Dependents Crawling not possible:")
                    print(e)
                    dependents_repo = []
                    
                    
                number_of_dependents = len(dependents_repos)
                dependents_repos = ",".join(dependents_repos)
                
                # Check if the project is still active
                closed_issues = repo.get_issues(state="closed")
                open_issues = repo.get_issues(state="open")

                closed_pullrequests = repo.get_pulls(state="closed")
                open_pullrequests = repo.get_pulls(state="open")
                
                closed_prs = closed_pullrequests.totalCount
                open_prs = open_pullrequests.totalCount
                
                if closed_prs > 10:
                    pr_review_analyse = 10
                else:
                    pr_review_analyse = closed_prs
                
                total_reviews = 0
                for pull_request in closed_pullrequests[0:pr_review_analyse-1]:
                    pr_reviews = pull_request.get_reviews()
                    total_reviews = total_reviews + pr_reviews.totalCount
                
                try:
                    reviews_per_pr = total_reviews/pr_review_analyse
                    
                except:                   
                    reviews_per_pr = 0

                if closed_issues.totalCount > 0:
                    last_issue_closed = closed_issues[0].updated_at

                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_closed_timedelta = datetime.utcnow() - last_issue_closed

                if (
                    last_issue_closed_timedelta > timedelta(days=365)
                    and total_commits_last_year == 0
                    or repo.archived
                ):
                    print("%s is an inactive project" % entry.name)
                    project_active_checkbox = "false"
                else:
                    print("%s is an active project" % entry.name)
                    project_active_checkbox = "true"

                try:
                    license = repo.get_license()
                    if license.license.spdx_id == "NOASSERTION":
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"

                labels = ",".join([entry.name for entry in repo.get_labels()])
                topics = ",".join(repo.get_topics())

                languages_states = repo.get_languages()
                programming_languages = ",".join(languages_states.keys())
                
                try:
                    dominating_language = list(languages_states.keys())[0]
                except:
                    dominating_language = ""

                try:
                    dotfolder_file = repo.get_contents(".github")
                    for file in dotfolder_file:
                        if file.path == ".github/FUNDING.yml":
                            print("Funding found")
                            donation_platforms = base64.b64decode(file.content)
                            accepts_donations = "true"
                        else:
                            donation_platforms = None
                            accepts_donations = "false"
                except:
                    print("No funding information found")
                    donation_platforms = None
                    accepts_donations = "false"

                try:
                    code_of_conduct = "false"
                    contribution_guide = "false"
                    for file_content in contents_root:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                    for file_content in dotfolder_file:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                except Exception as e:
                    print(e)

                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(
                    sorted(contributor_activity.items(), key=lambda item: item[1])
                )
                weighted_contribution = {
                    k: v / commits_total for k, v in sorted_contributor.items()
                }

                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(
                    weighted_contribution.values()
                )

                try:
                    last_released_date = releases[0].published_at.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    )
                    last_release_tag_name = releases[0].tag_name

                except Exception as e:
                    print("No Release found")
                    last_released_date = ""
                    last_release_tag_name = ""
                    print(e)

                total_number_of_commits = commits.totalCount
                
                stars_last_year = 0
                for star in stargazers:
                    starred_delta = datetime.utcnow() - star.starred_at
                    if  starred_delta < timedelta(days=365):
                        stars_last_year = stars_last_year + 1

                # Gathering organization data
                if repo.organization is None:
                    organization_user_name = None

                elif repo.organization.login not in organization_list:
                    organization_user_name = repo.organization.login
                    organization_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    organization_public_repos = repo.organization.public_repos
                    organization_location = repo.organization.location
                    organization_github_url = repo.organization.html_url
                    organization_website = repo.organization.blog
                    organization_created = repo.organization.created_at.strftime("%Y/%m/%d, %H:%M:%S")
                    organization_repos = g.search_repositories(query='user:'+organization_user_name, sort='updated')     
                    organization_last_update = organization_repos[0].updated_at.strftime("%Y/%m/%d, %H:%M:%S")
                        
                        
                    organization_data = {
                        "organization_name": organization_name,
                        "organization_user_name":organization_user_name,
                        "organization_github_url":organization_github_url,
                        "organization_website":organization_website,
                        "organization_avatar": organization_avatar,
                        "organization_location": organization_location,
                        "organization_public_repos": organization_public_repos,
                        "organization_created": organization_created,
                        "organization_last_update": organization_last_update
                    }

                    organization_list.append(organization_user_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)

                else:
                    organization_name = repo.organization.login
                    organization_user_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    

                project_data = {
                    "project_name": entry.name,
                    "git_namespace": user,
                    "git_url": repo.clone_url,
                    "rubric": r.key,
                    "oneliner": entry.text[2:],
                    "topics": topics,
                    "organization": organization_name,
                    "organization_user_name": organization_user_name,
                    "created": repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "last_update": repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "total_commits_last_year": total_commits_last_year,
                    "project_active_checkbox": project_active_checkbox,
                    "last_issue_closed": last_issue_closed.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    ),
                    "last_released_date": last_released_date,
                    "last_release_tag_name": last_release_tag_name,
                    "total_number_of_commits": total_number_of_commits,
                    "community_development_distribution_score": community_development_distribution_score,
                    "stargazers_count": repo.stargazers_count,
                    "number_of_dependents": number_of_dependents,
                    "dependents_repos": dependents_repos,
                    "stars_last_year": stars_last_year,
                    "dominating_language": dominating_language,
                    "languages": programming_languages,
                    "homepage": repo.homepage,
                    "closed_issues": closed_issues.totalCount,
                    "open_issues": open_issues.totalCount,
                    "closed_pullrequests": closed_prs,
                    "open_pullrequests": open_prs,
                    "reviews_per_pr": reviews_per_pr,
                    "good_first_issue": repo.get_issues(
                        state="open", labels=["good first issue"]
                    ).totalCount,
                    "license": license_name,
                    "contributors": repo.get_contributors().totalCount,
                    "accepts_donations": accepts_donations,
                    "donation_platforms": donation_platforms,
                    "code_of_conduct": code_of_conduct,
                    "contribution_guide": contribution_guide,
                    "organization_avatar":organization_avatar
                }

                print("Project Data:")
                print(project_data)
                writer_github_projects.writerow(project_data)

            except Exception as e:
                print(colored("Failed to gather project information:"))
                print(colored(e, "red"))
                print(traceback.format_exc())

        elif urlparse(entry.url).netloc == "gitlab.com":
            print("%s is a Gitlab project" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
            }
            print("Project Data:")
            print(project_data)
            writer_gitlab_projects.writerow(project_data)

        else:
            print("%s is hosted on custom platform" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
            }

            print("Project Data:")
            print(project_data)
            writer_custom_platform_projects.writerow(project_data)

csv_github_projects.close()
csv_github_organizations.close()
csv_gitlab_projects.close()
csv_custom_platform_projects.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 749
pvlib-python is an active project
Code of conduct found
Contribution guide found
Organization Data:
{'organization_name': 'pvlib', 'organization_user_name': 'pvlib', 'organization_github_url': 'https://github.com/pvlib', 'organization_website': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11037261?v=4', 'organization_location': None, 'organization_public_repos': 4, 'organization_created': '2015/02/17, 00:04:09', 'organization_last_update': '2021/11/22, 10:16:29'}
Project Data:
{'project_name': 'pvlib-python', 'git_namespace': 'pvlib', 'git_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renewable-energy,renewable

{'project_name': 'pvfactors', 'git_namespace': 'SunPower', 'git_url': 'https://github.com/SunPower/pvfactors.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Open source view-factor model for diffuse shading and bifacial PV modeling.', 'topics': 'solar-energy,renewable-energy,python,bifacial', 'organization': 'SunPower', 'organization_user_name': 'SunPower', 'created': '2018/05/14, 06:10:55', 'last_update': '2021/10/29, 19:53:01', 'total_commits_last_year': 8, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/29, 19:53:01', 'last_released_date': '2021/03/27, 13:39:44', 'last_release_tag_name': 'v1.5.1', 'total_number_of_commits': 99, 'community_development_distribution_score': 0.09090909090909094, 'stargazers_count': 50, 'number_of_dependents': 5, 'dependents_repos': 'thesethtruth/LESO,prathamrg/solarapp,anomam/pvfactors_iea_pvps_study,toddkarin/vocmax,toddkarin/pvtools', 'stars_last_year': 11, 'dominating_language': 'Python', 'languages': 'Python', 'homepa

{'project_name': 'Machine-Learning-for-Solar-Energy-Prediction', 'git_namespace': 'ColasGael', 'git_url': 'https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Predict the power production of a solar panel farm from weather measurements using machine learning.', 'topics': 'machine-learning,neural-network,data-processing,python,matlab,tensorflow', 'organization': 'National Renewable Energy Laboratory', 'organization_user_name': None, 'created': '2018/05/06, 19:43:04', 'last_update': '2021/11/18, 12:17:18', 'total_commits_last_year': 0, 'project_active_checkbox': 'false', 'last_issue_closed': '2019/11/15, 19:01:02', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 11, 'community_development_distribution_score': 0.0, 'stargazers_count': 117, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 36, 'dominating_language': 'Python', 'languages': 'Python,MATLAB

{'project_name': 'pvcaptest', 'git_namespace': 'pvcaptest', 'git_url': 'https://github.com/pvcaptest/pvcaptest.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Collection of functions and Jupyter Notebooks to partially automate running a capacity test following ASTM E2848.', 'topics': '', 'organization': None, 'organization_user_name': 'pvcaptest', 'created': '2017/09/19, 01:33:23', 'last_update': '2021/07/26, 13:43:32', 'total_commits_last_year': 37, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/07/25, 16:51:03', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 606, 'community_development_distribution_score': 0.0018832391713747842, 'stargazers_count': 12, 'number_of_dependents': 1, 'dependents_repos': 'pvcaptest/pvcaptest', 'stars_last_year': 2, 'dominating_language': 'Python', 'languages': 'Python', 'homepage': None, 'closed_issues': 39, 'open_issues': 28, 'closed_pullrequests': 15, 'open_pullrequests': 2, 'reviews_per_pr

{'project_name': 'SolarPV-DER-simulation-utility', 'git_namespace': 'tdcosim', 'git_url': 'https://github.com/tdcosim/SolarPV-DER-simulation-tool.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Allows user to run dynamics simulations for solar photovoltaic distributed energy resource connected to a stiff voltage source or to an external program.', 'topics': '', 'organization': 'TDcoSim Team', 'organization_user_name': 'tdcosim', 'created': '2019/03/19, 21:59:40', 'last_update': '2021/09/16, 21:02:00', 'total_commits_last_year': 41, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/09/09, 05:07:30', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 205, 'community_development_distribution_score': 0.014925373134328401, 'stargazers_count': 20, 'number_of_dependents': 1, 'dependents_repos': 'tdcosim/TDcoSim', 'stars_last_year': 3, 'dominating_language': 'Python', 'languages': 'Python', 'homepage': '', 'closed_issues': 26, 'open_iss

...

{'project_name': 'awesome-earth-engine-apps', 'git_namespace': 'philippgaertner', 'git_url': 'https://github.com/philippgaertner/awesome-earth-engine-apps.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'An awesome list of all available Google Earth Engine Apps and user-specific App Galleries.', 'topics': 'google-earth-engine,earth-engine,apps', 'organization': 'mySociety', 'organization_user_name': None, 'created': '2021/05/28, 14:41:43', 'last_update': '2021/11/23, 10:58:13', 'total_commits_last_year': 92, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/19, 11:39:43', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 92, 'community_development_distribution_score': 0.0, 'stargazers_count': 58, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 58, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 0, 'open_issues': 0, 'closed_pullrequests': 0, 'open_pullrequests': 0, 'reviews_per_

{'project_name': 'Awesome Green Software', 'git_namespace': 'Green-Software-Foundation', 'git_url': 'https://github.com/Green-Software-Foundation/awesome-green-software.git', 'rubric': 'Curated Lists', 'oneliner': 'Research, tools, code, libraries and training to for building applications that emit less carbon into our atmosphere.', 'topics': 'green-software,sustainable-software,sustainability,awesome-list,green-ai,sustainable-ai,climate,green-computing,hacktoberfest', 'organization': 'Green Software Foundation', 'organization_user_name': 'Green-Software-Foundation', 'created': '2021/07/18, 21:52:25', 'last_update': '2021/11/23, 04:45:26', 'total_commits_last_year': 6, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/10, 13:50:21', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 9, 'community_development_distribution_score': 0.6666666666666667, 'stargazers_count': 36, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year':

{'project_name': 'awesome-transit', 'git_namespace': 'CUTR-at-USF', 'git_url': 'https://github.com/CUTR-at-USF/awesome-transit.git', 'rubric': 'Curated Lists', 'oneliner': 'Community list of transit APIs, apps, datasets, research, and software.', 'topics': 'transit,gtfs,awesome-list,bus,datasets,awesome,list,tools,transit-data,gtfs-feed,gtfs-realtime,gtfs-validator,gtfs-files,transit-map,transit-agencies,realtime-data,gtfs-utils,gtfs-libraries,gtfs-converters,gtfs-analysis', 'organization': 'Center for Urban Transportation Research @ USF', 'organization_user_name': 'CUTR-at-USF', 'created': '2015/08/13, 00:38:53', 'last_update': '2021/11/25, 10:14:48', 'total_commits_last_year': 46, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/10, 17:53:18', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 364, 'community_development_distribution_score': 0.4199134199134199, 'stargazers_count': 779, 'number_of_dependents': 0, 'dependents_repos': '', '

{'project_name': 'Cryosphere Software, Data and Tools', 'git_namespace': 'awesome-cryosphere', 'git_url': 'https://github.com/awesome-cryosphere/cryosphere-links.git', 'rubric': 'Curated Lists', 'oneliner': 'A prototype of a curated list of awesome data sources, models, tools and organizations related to the Cryosphere and its subspheres.', 'topics': 'cryosphere,glaciers,sea-ice,snow,permafrost,ice-sheets,awesome-list', 'organization': 'Awesome Cryosphere', 'organization_user_name': 'awesome-cryosphere', 'created': '2021/05/25, 13:33:46', 'last_update': '2021/11/07, 19:17:40', 'total_commits_last_year': 32, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/05, 07:35:53', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 40, 'community_development_distribution_score': 0.631578947368421, 'stargazers_count': 50, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 50, 'dominating_language': '', 'languages': '', 'homepage': ''

{'project_name': 'Awesome Open Geoscience', 'git_namespace': 'softwareunderground', 'git_url': 'https://github.com/softwareunderground/awesome-open-geoscience.git', 'rubric': 'Curated Lists', 'oneliner': 'Curated from repositories that make our lives as geoscientists, hackers and data wranglers easier or just more awesome.', 'topics': 'awesome-list,geoscience,open-source,open-data,awesome,geophysics,reservoir-modeling,seismic,geospatial,subsurface,geology,open-science,python,cheatsheet,geosciences,geostatistics,simulation,geochemistry,groundwater,modflow', 'organization': 'Software Underground', 'organization_user_name': 'softwareunderground', 'created': '2017/10/18, 13:39:37', 'last_update': '2021/11/26, 11:41:04', 'total_commits_last_year': 33, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/16, 08:51:12', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 405, 'community_development_distribution_score': 0.7037037037037037, 'stargazers_